# **IP4_Run2**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [ ]:
problem = """Imagine you own a small shop, and you're trying to manage your employees' working hours and sales targets efficiently.
You have three kinds of goals overall: sales goals, employee hours goals and overtime goals.

Sales Goals:
You have a target of selling 5500 units.
But sometimes, you might sell fewer (negative deviation) or more (positive deviation) than this target.
On average, your full-time employees will sell 5 records per hour and your part-time employees will sell 2 records per hour.

Employee Hours:
You have both full-time and part-time employees.
Your full-time employees are expected to work a total of 800 hours, while part-time employees are expected to work 320 hours.
Similarly, like sales, there could be deviations in these working hours.

Overtime:
Additionally, your employees might end up working overtime.
You have a target of 100 hours of overtime for full-time employees.
You are not concerned about limiting the overtime of part-time employees.
Again, you might deviate from this goal.

Each of these deviations has a different priority for you.
P1 to P4 are priority factors for each deviation.
Your most important goal is to achieve your sales goal.
Second to that is the goal to not exceed 100 overtime hours for full-time employees.
Then, you next priority is to fully utilize your full-time and part time employees.
Your last goal with the lowest priority, you want to keep general overtime hours for both kinds of employees as low as possible.
In addition to the general priorities of you goals, you are twice as concerned about fully utilizing the full-time employee hours.
Finally, overtime hours of part-time employees have a relative cost of three times that of full-time employees.

Now, the goal here is to minimize the overall impact of these deviations, considering their priorities.
This means finding the best balance between sales targets, employee hours, and overtime, while keeping in mind which deviations are more critical for your business. """

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [ ]:
print(response1.choices[0].message.content)

Parameters:
1. P1 - Priority factor for deviation from sales target.
2. P2 - Priority factor for deviation from overtime target for full-time employees.
3. P3 - Priority factor for deviation from working hours for full-time and part-time employees.
4. P4 - Priority factor for deviation from general overtime hours.
5. s - Sales target in units.
6. h_full - Total expected hours of work for full-time employees.
7. h_part - Total expected hours of work for part-time employees.
8. o_full - Overtime hours target for full-time employees.
9. r_full - Full-time employees' records selling rate per hour. 
10. r_part - Part-time employees' records selling rate per hour.

Decision Variables: 
1. X_full - Actual working hours of full-time employee, including overtime.
2. X_part - Actual working hours of part-time employee, including overtime.
3. Yo_full - Overtime hours by full-time employees.
4. Yo_part - Overtime hours by part-time employees.

Objective Function:
To minimize: 
P1*(s - r_full*X_ful

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [ ]:
Markdown(response2.choices[0].message.content)

Python Pyomo Code for Above Optimization Problem:

```python
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define Decision Variables
model.x_full = Var(within=NonNegativeReals)
model.x_part = Var(within=NonNegativeReals)
model.yo_full = Var(within=NonNegativeReals)
model.yo_part = Var(within=NonNegativeReals)

# Sample Data
P1 = 2
P2 = 3
P3 = 1
P4 = 4
s = 5000
h_full = 40
h_part = 20
o_full = 5
r_full = 50
r_part = 30

# Objective Function
model.obj = Objective(
    expr=(P1*(s - r_full*model.x_full - r_part*model.x_part))**2 +
    (P2*(o_full - model.yo_full))**2 + 2*P3*(h_full - model.x_full)**2 + P3*(h_part - model.x_part)**2 + 
    P4*(model.yo_full + 3*model.yo_part)**2, sense=minimize)

# Constraint Definitions
def full_overtime_constraint(model):
    return model.yo_full >= model.x_full - h_full

model.full_overtime_con = Constraint(rule=full_overtime_constraint)

def part_overtime_constraint(model):
    return model.yo_part >= model.x_part - h_part

model.part_overtime_con = Constraint(rule=part_overtime_constraint)

# Create a solver
solver = SolverFactory('ipopt')

# Solve the model
solver.solve(model)

# Print the results
print("Results of the Optimization Problem:")
print('Full Time Working Hours:', value(model.x_full))
print('Part Time Working Hours:', value(model.x_part))
print('Full Time Overtime Hours:', value(model.yo_full))
print('Part Time Overtime Hours:', value(model.yo_part))
```
In this code, 'ipopt' is used as solver. Any other solver could be used depending on requirement. Sample data is inputted as per convenience, could be replaced with actual dataset.

### **Run the code Generated by GPT4**

In [ ]:
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid="5bc31ed5-340e-4421-b552-385179b7e034",  # license to use
)  # instantiate AMPL object and register magics

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [47]:
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define Decision Variables
model.x_full = Var(within=NonNegativeReals)
model.x_part = Var(within=NonNegativeReals)
model.yo_full = Var(within=NonNegativeReals)
model.yo_part = Var(within=NonNegativeReals)

# data inputted by human
P1 = 4
P2 = 3
P3 = 2
P4 = 1
s = 5500
h_full = 800
h_part = 320
o_full = 100
r_full = 5
r_part = 2

# Objective Function
model.obj = Objective(
    expr=(P1*(s - r_full*model.x_full - r_part*model.x_part))**2 +
    (P2*(o_full - model.yo_full))**2 + 2*P3*(h_full - model.x_full)**2 + P3*(h_part - model.x_part)**2 +
    P4*(model.yo_full + 3*model.yo_part)**2, sense=minimize)

# Constraint Definitions
def full_overtime_constraint(model):
    return model.yo_full >= model.x_full - h_full

model.full_overtime_con = Constraint(rule=full_overtime_constraint)

def part_overtime_constraint(model):
    return model.yo_part >= model.x_part - h_part

model.part_overtime_con = Constraint(rule=part_overtime_constraint)

# Create a solver
solver = SolverFactory('knitro')
# Solve the model
solver.solve(model)

# Print the results
print("Results of the Optimization Problem:")
print('Full Time Working Hours:', value(model.x_full))
print('Part Time Working Hours:', value(model.x_part))
print('Full Time Overtime Hours:', value(model.yo_full))
print('Part Time Overtime Hours:', value(model.yo_part))
print(model.obj())

Results of the Optimization Problem:
Full Time Working Hours: 965.5300155104964
Part Time Working Hours: 327.18143299785123
Full Time Overtime Hours: 165.5300155110596
Part Time Overtime Hours: 7.181432998667848
188524.88284044372


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**